<a href="https://colab.research.google.com/github/Decoding-Data-Science/airesidency/blob/main/crewai_multiagent_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Multi Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [4]:
#setting the model
import os

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [5]:
import os
from google.colab import userdata

# Retrieve API key from Colab secrets and set it in environment variables
os.environ["OPENAI_API_KEY"] = userdata.get('openai')

In [6]:
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

In [7]:
# Set up API keys - enter from https://serper.dev/billing
os.environ["SERPER_API_KEY"] = "218873db9b73d345d3083e2f1f29cb74b14d4ddb" # serper.dev API key

# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

In [8]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic."    ,
    tools=[search_tool, web_rag_tool,docs_tool,file_tool],
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [9]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic in 1500 words: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [10]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [12]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [13]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.1500 words",
    agent=writer,
)

### Task: Edit

In [14]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [15]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    planning=True,
    verbose=4
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [16]:
result = crew.kickoff(inputs={"topic": "How close are we to Artificial General Intelligence ?"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on How close are we to Artificial General Intelligence ?.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I need to gather information on the latest trends, key players, and noteworthy news related to Artificial General Intelligence to prioritize them in the content plan.

Action: Search the internet
Action Input: {"search_query": "latest trends in Artificial General Intelligence"} 


Search results: Title: 6 AI trends you'll see more of in 2025 - Microsoft News
Link: https://news.microsoft.com/source/features/ai/6-ai-trends-youll-see-more-of-in-2025/
Snippet: In 2025, AI will evolve from a tool for work and home to an integral 

- Display the results of your execution as markdown in the notebook.

In [17]:
from IPython.display import Markdown
Markdown(result)

# How Close Are We to Achieving Artificial General Intelligence?

## Introduction
Artificial General Intelligence (AGI) has long been a goal for scientists and tech enthusiasts around the world. The concept of creating a machine that can perform any intellectual task that a human can do has sparked endless debates and discussions. In this article, we will delve into the latest trends, key players, and noteworthy news in the field of AGI. We will also explore the target audience for AGI content and the implications of its development on society and ethics.

## Latest Trends in Artificial General Intelligence
The evolution of AI from a mere tool to an essential part of our daily lives has been remarkable. AI-powered agents are becoming increasingly autonomous, simplifying tasks and improving efficiency in various industries. Breakthroughs in AI technology are shaping the future of AGI, bringing us closer to achieving this elusive goal.

Key players in the field of AGI, such as Apple, Microsoft, NVIDIA, Alphabet, and leading researchers like Geoff Hinton, Yann LeCun, and Yoshua Bengio, are pushing the boundaries of what is possible in artificial intelligence. Their contributions to the development of AGI are paving the way for a future where intelligent machines can think, learn, and adapt like humans.

## Noteworthy News on Artificial General Intelligence
Recent developments in the world of AGI have been nothing short of groundbreaking. The SoftBank chief's prediction of achieving Artificial Superintelligence (ASI) within the next decade has raised eyebrows and sparked discussions about the implications of such a technological advancement. In Albania, an AI minister is aiming to use AI to eliminate corruption, showcasing the potential of AGI to bring about positive societal change.

The implications of AGI development on society and ethics cannot be understated. As we move closer to achieving Artificial General Intelligence, questions about job displacement, privacy concerns, and ethical considerations are becoming more prevalent. It is crucial for us to address these issues proactively and ensure that the benefits of AGI outweigh the potential risks.

## Target Audience for AGI Content
Marketers, researchers, tech enthusiasts, and AI professionals are among the primary target audience for AGI content. Business owners looking to leverage AI for growth and efficiency can benefit from staying informed about the latest trends and developments in the field. The general audience seeking insights into the future of technology will also find value in learning about the potential of AGI and its impact on society.

As we navigate the complex landscape of Artificial General Intelligence, it is essential to stay informed and engaged with the latest developments. Join us on this journey towards achieving AGI and discover the endless possibilities it holds for the future.

In conclusion, the road to achieving Artificial General Intelligence is paved with challenges and opportunities. With the contributions of top players in the field, the latest trends in AI technology, and the dedication of researchers and tech enthusiasts worldwide, we are closer than ever to realizing the dream of creating machines that can think and learn like humans. As we continue on this journey, it is crucial to consider the ethical implications and societal impact of AGI development to ensure a future where intelligent machines coexist harmoniously with humanity.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "Chatgpt Agent by openAI"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Chatgpt Agent by openAI.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I need to gather information on the latest trends, key players, and noteworthy news on Chatgpt Agent by openAI to create an engaging and informative blog article.

Action: Search the internet
Action Input: {"search_query": "latest trends Chatgpt Agent by openAI"} 


Search results: Title: Introducing ChatGPT agent: bridging research and action - OpenAI
Link: https://openai.com/index/introducing-chatgpt-agent/
Snippet: ChatGPT now thinks and acts, proactively choosing from a toolbox of agentic skills to complete tasks for you using its own computer.
---
T

In [ ]:
from IPython.display import Markdown
Markdown(result)

# Unleashing the Power of ChatGPT Agent by OpenAI: A Comprehensive Guide

ChatGPT Agent by OpenAI has made a significant impact on the artificial intelligence landscape, introducing cutting-edge features and capabilities that have captured the attention of tech companies, researchers, developers, and organizations of all sizes. With its ability to control a computer, recent rollout for $20 Plus users, and advanced functionalities, ChatGPT Agent is paving the way for the future of AI technology.

The launch of ChatGPT Agent has sparked excitement in the tech industry, showcasing new features and enhancements that solidify its position as a leading AI technology. The team behind OpenAI's ChatGPT Agent has demonstrated expertise and dedication to innovation, pushing the boundaries of what AI can achieve. News of its ability to control a computer has particularly resonated with businesses seeking to streamline processes and increase productivity.

ChatGPT Agent caters to a diverse audience, including tech companies, researchers, developers, and organizations of all sizes. Businesses looking to enhance workflows and productivity can leverage ChatGPT Agent effectively. Furthermore, individuals interested in AI technology and its applications will find ChatGPT Agent to be a valuable tool for exploring the possibilities of artificial intelligence.

ChatGPT Agent offers a wide array of features that differentiate it from other AI technologies. Its advanced AI agent capabilities can assist users in various tasks, from answering questions to generating creative content. The virtual assistant features make it a valuable tool for automation and productivity enhancement, enabling businesses to streamline operations and improve efficiency.

The applications of ChatGPT Agent extend across industries and sectors, providing real-world benefits. Tech companies can enhance customer service with instant support, while researchers can expedite data analysis and research processes. Developers can create interactive experiences by integrating ChatGPT Agent into their applications, showcasing its versatility and adaptability.

While the benefits of using ChatGPT Agent are evident, challenges such as data security and ethical considerations must be addressed when integrating AI technology. It is crucial for businesses to weigh these factors before implementing ChatGPT Agent into their operations. Looking ahead, ChatGPT Agent is poised to become an essential tool as AI technology progresses, offering innovative solutions for businesses and individuals alike.

In conclusion, ChatGPT Agent by OpenAI is a transformative AI technology that is reshaping interactions with machines and task automation. With its advanced capabilities, real-world applications, and potential advantages, ChatGPT Agent is a valuable asset for those in the tech industry, researchers, developers, and anyone interested in AI technology. Embracing the future of AI innovation, ChatGPT Agent leads the way towards a smarter and more efficient future.

Take Action: Explore ChatGPT Agent by OpenAI further and unlock the potential of this advanced AI technology. Businesses seeking enhanced productivity should consider integrating ChatGPT Agent into their operations. For more information on ChatGPT Agent and its capabilities, visit OpenAI's official website and reputable sources.

In [ ]:
Markdown(result)

# Deploy this crew in gradio

In [18]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.6/46.6 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.9.4
    Uninstalling typer-0.9.4:
      Successfully uninstalled typer-0.9.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instructor 0.5.2 requires typer<0.10.0,>=0.9.0, but you have typer 0.17.4 which is incompatible.


In [19]:
import gradio as gr
from crewai import Agent, Task, Crew

# Define Agents
researcher = Agent(
    role="Senior Research Specialist",
    goal="Uncover intricate insights into the given topic",
    backstory="A seasoned expert in extracting valuable information and providing detailed analysis on any given subject.",
    allow_delegation=False
)

writer = Agent(
    role="Content Writer",
    goal="Craft engaging and informative articles",
    backstory="An experienced content writer who can turn complex research into easy-to-understand and engaging articles.",
    allow_delegation=False
)

editor = Agent(
    role="Content Editor",
    goal="Ensure articles are polished, coherent, and engaging",
    backstory="An expert editor with a sharp eye for clarity, grammar, and flow, capable of turning drafts into publication-ready articles.",
    allow_delegation=False
)

# Function to setup tasks and run CrewAI
def run_crewai_article_writer(topic):
    # Define tasks
    research_task = Task(
        description=f"Conduct in-depth research on the topic: {topic} and provide a structured summary.",
        expected_output="A structured and detailed research summary covering important aspects, recent developments, and insights related to the topic.",
        agent=researcher
    )

    writing_task = Task(
        description=f"Using the research, write a full-length article on the topic: {topic} in 1500 words give various sub section",
        expected_output="An engaging, detailed, informative, and coherent article suitable for online publication.",
        agent=writer
    )

    editing_task = Task(
        description=f"Review and edit the article on {topic} for clarity, grammar, flow, and engagement.",
        expected_output="A polished, clear, and professionally edited final version of the article.",
        agent=editor
    )

    # Setup Crew
    crew = Crew(
        agents=[researcher, writer, editor],
        tasks=[research_task, writing_task, editing_task],
        verbose=True  # Set to False if you want less console output
    )

    # Execute
    result = crew.kickoff()
    return result

# Gradio Interface
def generate_article(topic):
    return run_crewai_article_writer(topic)

iface = gr.Interface(
    fn=generate_article,
    inputs=gr.Textbox(lines=2, placeholder="Enter a topic for the article..."),
    outputs=gr.Textbox(lines=20, label="Generated Article"),
    title="DDS CrewAI Article Writer with Researcher, Writer, and Editor",
    description="Enter a topic and let CrewAI agents research, write, and edit a complete article!"
)

if __name__ == "__main__":
    iface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://91fe87c3d58ed07138.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import os
from google.colab import files
import shutil

# Step 1: Create target directory
upload_dir = "blog-posts"
os.makedirs(upload_dir, exist_ok=True)
print(f"✅ Directory '{upload_dir}' is ready.")

✅ Directory 'blog-posts' is ready.


In [ ]:


# Step 2: Upload file(s)
uploaded_files = files.upload()

# Step 3: Move uploaded files to the target folder
for filename in uploaded_files.keys():
    shutil.move(filename, os.path.join(upload_dir, filename))
    print(f"📥 Moved '{filename}' to '{upload_dir}/'")

print("✅ All files uploaded and saved.")


Saving Social Media Guidelines for UAE gov 2021 En.pdf to Social Media Guidelines for UAE gov 2021 En.pdf
📥 Moved 'Social Media Guidelines for UAE gov 2021 En.pdf' to 'blog-posts/'
✅ All files uploaded and saved.


In [ ]:
# Already executed in Colab
docs_tool = DirectoryReadTool(directory='blog-posts')


In [ ]:
from crewai_tools import DirectoryReadTool

# Tool setup (ensure directory exists)
docs_tool = DirectoryReadTool(directory='blog-posts')

# Agent using uploaded documents
researcher = Agent(
    role="Senior Research Specialist",
    goal="Uncover insights using uploaded documents and online sources",
    backstory="You're an expert in using both uploaded files and web search to produce comprehensive insights.",
    tools=[docs_tool],  # ← Enable document access
    allow_delegation=False
)


In [ ]:
crew = Crew(
    agents=[researcher],
    tasks=[research_task],
    verbose=True
)

result = crew.kickoff(inputs={"topic": "AI Regulation"})
print(result)
